In [1]:
from sqlalchemy import create_engine, exc
import pandas as pd

In [2]:
usuario = 'root'
senha = 'root'
host = 'localhost'
portorigem = 3306
portdestino = 3306
bancoorigem = 'vendas'
bancodestino = 'vendas_dimensional'

In [3]:
try:
    engineori = create_engine(f"mysql+pymysql://{usuario}:{senha}@{host}:{portorigem}/{bancoorigem}")
    with engineori.connect() as connori:
        print(f"Conexão com o banco '{bancoorigem}' estabelecida com sucesso.")
except exc.SQLAlchemyError as e:
    print(f"Erro ao conectar ao banco '{bancoorigem}': {e}")
    exit()

Conexão com o banco 'vendas' estabelecida com sucesso.


In [4]:
try:
    enginedes = create_engine(f"mysql+pymysql://{usuario}:{senha}@{host}:{portdestino}/{bancodestino}")
    with enginedes.connect() as conndes:
        print(f"Conexão com o banco '{bancodestino}' estabelecida com sucesso.")
except exc.SQLAlchemyError as e:
    print(f"Erro ao conectar ao banco '{bancodestino}': {e}")
    exit()

Conexão com o banco 'vendas_dimensional' estabelecida com sucesso.


In [5]:
sql = f"""
select it.quantidade,it.precounitario,
    it.quantidade * it.precounitario * pe.frete as valorfrete,
    it.quantidade * it.precounitario * pe.desconto as valordesconto,
    it.quantidade * it.precounitario * pe.comissao as valorcomissao,
    cl.id as idcliente,
    pr.id as idproduto,
    ca.id as idcategoria,
    fo.id as idfornecedor,
    tr.id as idtransportadora,
    ve.id as idvendedor,
    pe.id as idpedido,
    pe.datadopedido,pe.datadopagamento,pe.previsaoentrega,
    pe.datadeenvio,pe.datadaentrega,
    pe.paisdestino,
    cl.nacionalidade as nacionalidadecliente,
    cl.pais as moradiacliente,
    pa1.sigla as paisfornecedor,
    pa2.sigla as paistransportadora,
    ve.pais as paisvendedor
from itens it
inner join pedidos         pe on pe.id = it.idpedido
inner join clientes        cl on cl.id = pe.idcliente 
inner join produtos        pr on pr.id = it.idproduto
inner join categorias      ca on ca.id = pr.idcategoria
inner join fornecedores    fo on fo.id = pr.idfornecedor
inner join transportadoras tr on tr.id = pe.via
inner join vendedores      ve on ve.id = pe.idvendedor
inner join paises          pa1 on pa1.id = fo.pais
inner join paises          pa2 on pa2.id = tr.pais
    """
itens = pd.read_sql(sql,engineori)
itens['paisdestino'] = itens['paisdestino'].fillna(itens['moradiacliente'])

In [6]:
sql = f"select id,data from dim_tempo"
dim_tempo = pd.read_sql(sql,enginedes)
dim_tempo['data'] = pd.to_datetime(dim_tempo['data'])

In [7]:
itens['datadopedido'] = pd.to_datetime(itens['datadopedido']).dt.normalize()
itens = pd.merge(itens,dim_tempo,left_on='datadopedido',right_on='data',how='left')
itens = itens.rename({'id':'idtempopedido'},axis=1)
itens = itens.drop(['datadopedido','data'],axis=1)

In [8]:
itens['datadopagamento'] = pd.to_datetime(itens['datadopagamento']).dt.normalize()
itens = pd.merge(itens,dim_tempo,left_on='datadopagamento',right_on='data',how='left')
itens = itens.rename({'id':'idtempopagamento'},axis=1)
itens = itens.drop(['datadopagamento','data'],axis=1)

In [9]:
itens['previsaoentrega'] = pd.to_datetime(itens['previsaoentrega']).dt.normalize()
itens = pd.merge(itens,dim_tempo,left_on='previsaoentrega',right_on='data',how='left')
itens = itens.rename({'id':'idtempoprevisao'},axis=1)
itens = itens.drop(['previsaoentrega','data'],axis=1)

In [10]:
itens['datadeenvio'] = pd.to_datetime(itens['datadeenvio']).dt.normalize()
itens = pd.merge(itens,dim_tempo,left_on='datadeenvio',right_on='data',how='left')
itens = itens.rename({'id':'idtempoenvio'},axis=1)
itens = itens.drop(['datadeenvio','data'],axis=1)

In [11]:
itens['datadaentrega'] = pd.to_datetime(itens['datadaentrega']).dt.normalize()
itens = pd.merge(itens,dim_tempo,left_on='datadaentrega',right_on='data',how='left')
itens = itens.rename({'id':'idtempoentrega'},axis=1)
itens = itens.drop(['datadaentrega','data'],axis=1)

In [12]:
sql = "select id,idtransacional from dim_cliente"
dim_cliente = pd.read_sql(sql,enginedes)

In [13]:
itens = pd.merge(itens,dim_cliente,left_on='idcliente',right_on='idtransacional',how='left')
itens = itens.drop(['idcliente','idtransacional'],axis=1)
itens = itens.rename({'id':'idcliente'},axis=1)

In [14]:
sql = "select id,idtransacional from dim_categoria"
dim_categoria = pd.read_sql(sql,enginedes)

In [15]:
itens = pd.merge(itens,dim_categoria,left_on='idcategoria',right_on='idtransacional',how='left')
itens = itens.drop(['idcategoria','idtransacional'],axis=1)
itens = itens.rename({'id':'idcategoria'},axis=1)

In [16]:
sql = "select id,idtransacional from dim_produto"
dim_produto = pd.read_sql(sql,enginedes)

In [17]:
itens = pd.merge(itens,dim_produto,left_on='idproduto',right_on='idtransacional',how='left')
itens = itens.drop(['idproduto','idtransacional'],axis=1)
itens = itens.rename({'id':'idproduto'},axis=1)

In [18]:
sql = "select id,idtransacional from dim_vendedor"
dim_vendedor = pd.read_sql(sql,enginedes)

In [19]:
itens = pd.merge(itens,dim_vendedor,left_on='idvendedor',right_on='idtransacional',how='left')
itens = itens.drop(['idvendedor','idtransacional'],axis=1)
itens = itens.rename({'id':'idvendedor'},axis=1)

In [20]:
sql = "select id,idtransacional from dim_transportadora"
dim_transportadora = pd.read_sql(sql,enginedes)

In [21]:
itens = pd.merge(itens,dim_transportadora,left_on='idtransportadora',right_on='idtransacional',how='left')
itens = itens.drop(['idtransportadora','idtransacional'],axis=1)
itens = itens.rename({'id':'idtransportadora'},axis=1)

In [22]:
sql = "select id,idtransacional from dim_fornecedor"
dim_fornecedor = pd.read_sql(sql,enginedes)

In [23]:
itens = pd.merge(itens,dim_fornecedor,left_on='idfornecedor',right_on='idtransacional',how='left')
itens = itens.drop(['idfornecedor','idtransacional'],axis=1)
itens = itens.rename({'id':'idfornecedor'},axis=1)

In [24]:
sql = "select id,idtransacional from dim_pedido"
dim_pedido = pd.read_sql(sql,enginedes)

In [25]:
itens = pd.merge(itens,dim_pedido,left_on='idpedido',right_on='idtransacional',how='left')
itens = itens.drop(['idpedido','idtransacional'],axis=1)
itens = itens.rename({'id':'idpedido'},axis=1)

In [26]:
sql = "select id,sigla from dim_pais"
dim_pais = pd.read_sql(sql,enginedes)

In [27]:
itens = pd.merge(itens,dim_pais,left_on='moradiacliente',right_on='sigla',how='left')
itens = itens.drop(['sigla','moradiacliente'],axis=1)
itens = itens.rename({'id':'idpaisorigemcliente'},axis=1)

In [28]:
itens = pd.merge(itens,dim_pais,left_on='nacionalidadecliente',right_on='sigla',how='left')
itens = itens.drop(['sigla','nacionalidadecliente'],axis=1)
itens = itens.rename({'id':'idpaisnacionalidadecliente'},axis=1)

In [29]:
itens = pd.merge(itens,dim_pais,left_on='paisdestino',right_on='sigla',how='left')
itens = itens.drop(['sigla','paisdestino'],axis=1)
itens = itens.rename({'id':'idpaisdestino'},axis=1)

In [30]:
itens = pd.merge(itens,dim_pais,left_on='paistransportadora',right_on='sigla',how='left')
itens = itens.drop(['sigla','paistransportadora'],axis=1)
itens = itens.rename({'id':'idpaistransportadora'},axis=1)

In [31]:
itens = pd.merge(itens,dim_pais,left_on='paisvendedor',right_on='sigla',how='left')
itens = itens.drop(['sigla','paisvendedor'],axis=1)
itens = itens.rename({'id':'idpaisvendedor'},axis=1)

In [32]:
itens = pd.merge(itens,dim_pais,left_on='paisfornecedor',right_on='sigla',how='left')
itens = itens.drop(['sigla','paisfornecedor'],axis=1)
itens = itens.rename({'id':'idpaisfornecedor'},axis=1)

In [33]:
itens = itens.reset_index()
itens = itens.rename({'index':'id','quantidade': 'qtdvendida','precounitario':'precounitarionavenda'},axis=1)
itens['id'] += 1

In [ ]:
try:
    itens.to_sql('fat_item',if_exists='append',index=False,con=enginedes)
    print("Fato carregado com sucesso.")
except exc.SQLAlchemyError as e:
        print(f"Erro ao carregar: {e}")